<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW0402.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>